Ako imamo diferencijabilnu funkciju koju hoćemo da minimizujemo možemo da koristimo metode gradijentnog spusta. Kako znamo da je gradijent vektor koji pokazuje pravac _lokalno_ najbržeg uspona, iterativno pravimo korake u smeru suprotnom gradijentu. Pošto je to samo lokalno najstrmiji pravac, ne pravimo velike korake (veličinu koraka kontrolišemo parametrom $\alpha$, poznat i kao learning rate).

Metod inercije (momentum) je unapređenje osnovnog gradijentnog spusta sa idejom da ako smo napravili više koraka u jednom smeru, verovatno treba tako i da nastavimo. Dakle, na trenutni korak utiču i prethodno napravljeni koraci. Ipak, nisu svi prethodni koraci jednako bitni - bitniji su nam skoriji koraci, nego oni od pre mnogo iteracija. Stoga koristimo parametar $\beta \in (0,1)$ da smanjimo važnost davno donesenih odluka.

Nesterovljev metod je gotovo isti kao metod inercije, samo što prvo pravimo "međukorak" samo na osnovu inercije (jer bismo taj deo svakako napravili), pa tek onda u toj novoj tački računamo gradijent, i, naravno, idemo u smeru suprotnom od njega.

Adam je jedan od najkorišćenijih algoritama u mašinskom učenju. Ideja je da na osnovu izgleda funkcije prilagodimo veličinu koraka.
Pravimo ocenu prvog i drugog statističkog momenta (očekivanje gradijenta i kvadrata gradijenta). Što se tiče prvog momenta logika je ista kao kod inercije. Drugi momenat nam govori kakve su promene gradijenata. Ako su promene velike, to znači da treba da usporimo, jer verovatno "skačemo" oko nekog minimuma. Ako su promene male, znači da ne menjamo mnogo pravac i možemo da ubrzamo, odnosno povećamo korak. Dakle, veličina koraka treba da bude proporcionalna prvom, a obrnuto proporcionalna drugom momentu.
Kao i obično, želimo da naše statističke ocene budu nepristrasne. Pokušajte da izvedete formulu za popravljanje ocena (u kodu m_hat i v_hat) koju smo koristili da bismo dobili nepristrasnost.

In [5]:
import numpy as np

In [ ]:
def f(x: np.ndarray):
    """Toy example function that is convex and has a global minimum at (0,0)"""
    return 0.5*(x[0]**2 + 10*x[1]**2)

In [ ]:
def gradient(x: np.ndarray):
    """Gradient of f"""
    return np.array([x[0], 10*x[1]])

In [ ]:
def alpha(i: int):
    """Calculates alpha (learning rate) based on iteration i"""
    # return 0.01 // i
    return 0.01

In [ ]:
def gd(x0: np.ndarray, num_iters: int, alpha_fn: callable, eps: float):
    """Basic gradient descent"""
    x = x0
    for it in range(num_iters):
        x_new = x - alpha_fn(it) * gradient(x)

        if abs(f(x) - f(x_new)) < eps:
            break

        x = x_new

    return x, it

In [11]:
gd(x0=np.array([2,5]), num_iters=1000, alpha_fn=alpha, eps=1e-5)

(array([3.15039757e-02, 6.32595944e-19]), 413)

In [ ]:
def momentum(x0: np.ndarray, num_iters: int, alpha_fn: callable, eps: float, beta: float):
    """Gradient descent with momentum - accumulate previous steps and account for them"""
    x = x0
    inertia = np.array([0,0])
    for it in range(num_iters):
        inertia = beta * inertia + alpha_fn(it) * gradient(x)
        x_new = x - inertia

        if abs(f(x) - f(x_new)) < eps:
            break

        x = x_new

    return x, it

In [ ]:
momentum(x0=np.array([2,5]), num_iters=1000, alpha_fn=alpha, eps=1e-5, beta=0.9)

(array([2.18161734e-02, 2.25643742e-33]), 220)

In [ ]:
def nesterov(x0: np.ndarray, num_iters: int, alpha_fn: callable, eps: float, beta: float):
    """
    Nesterov's gradient descent

    Basic idea: since we are going to move for inertia vector either way, let's do
    that first, and then calculate gradient at that point and, as usual, move
    in the opposite direction of the gradient.
    """
    x = x0
    inertia = np.array([0,0])
    for it in range(num_iters):
        inertia = beta * inertia + alpha_fn(it) * gradient(x - inertia)
        x_new = x - inertia

        if abs(f(x) - f(x_new)) < eps:
            break

        x = x_new

    return x, it

In [16]:
nesterov(x0=np.array([2,5]), num_iters=1000, alpha_fn=alpha, eps=1e-5, beta=0.9)

(array([0.00466766, 0.00275261]), 63)

In [ ]:
def adam(x0: np.ndarray, num_iters: int, alpha_fn: callable, eps: float, beta1: float, beta2: float, delta: float):
    """
    Adam - Adaptive moments estimation

    Idea: estimate first and second moment of the gradient and adapt your step based on them.
    If the first moment is large - we made several steps in the same direction - we should increase step size.
    If the second moment is large - we are making steps in different, opposite directions - we should decrease step size.
    Analogous reasoning stands for when the first and second moments are small.
    """
    x = x0
    m = np.array([0, 0])
    v = np.array([0, 0])
    for it in range(1, num_iters + 1):
        grad = gradient(x)
        # exponential moving average (EMA)
        m = beta1 * m + (1 - beta1) * grad
        v = beta2 * v + (1 - beta2) * grad**2

        # m and v are not unbiased estimates until the following step
        m_hat = m / (1 - beta1 ** it)
        v_hat = v / (1 - beta2 ** it)

        # sqrt(v) to make m and v be on the same scale, not grad and grad**2
        # +delta to avoid zero division
        x_new = x - alpha_fn(it) * m_hat / (np.sqrt(v_hat) + delta)

        if abs(f(x) - f(x_new)) < eps:
            break

        x = x_new

    return x, it

In [27]:
adam(x0=np.array([2,5]), num_iters=10000, alpha_fn=lambda x: 0.01, eps=1e-5, beta1=0.9, beta2=0.999, delta=1e-7)

(array([1.36156731e-16, 1.15471411e-02]), 1377)